In [ ]:
# Merge the two DataFrames (outer join to retain all rows)
import pandas as pd

joined_file_path = '../2_joined_data/2017_2024_GSM_joined.csv'

df1 = pd.read_csv("2017_GSM_parsed.csv", delimiter=",", header="infer")
print(df1.columns)
print(len(df1))
df2 = pd.read_csv("2020_GSM_parsed.csv", delimiter=",", header="infer")
print(df2.columns)
print(len(df2))
df3 = pd.read_csv("2023_GSM_parsed.csv", delimiter=",", header="infer")
print(df3.columns)
print(len(df3))
df4 = pd.read_csv("2024_GSM_parsed.csv", delimiter=",", header="infer")
print(df4.columns)
print(len(df4))

merged_df = pd.concat([df1, df2, df3, df4])
merged_df.drop(columns=["weight_oz", "weight"], inplace=True)
 # remove unnecessary cols
print(len(merged_df))

# Initialize a final DataFrame to store the cleaned rows
final_rows = []

# Group by the 'id' column to compare duplicates
for id, group in merged_df.groupby('id'):
    if len(group) == 1:
        # Case 1: Unique or Identical Row (only one row in group)
        final_rows.append(group.iloc[0])
    else:
        # Case 2: Multiple Rows (Duplicates found)
        combined_row = group.iloc[0].copy()  # Start with the first row
        conflicting_rows = []

        for _, row in group.iterrows():
            for col in row.index:
                if pd.isna(combined_row[col]) and not pd.isna(row[col]):
                    # Fill missing values
                    combined_row[col] = row[col]
                elif combined_row[col] != row[col] and not pd.isna(combined_row[col]) and not pd.isna(row[col]):
                    # Conflict detected
                    conflicting_rows.extend([combined_row.to_dict(), row.to_dict()])
                    print(f"{combined_row.id}, {col}: \n{combined_row[col]}\n{row[col]}\n")
        # print if there were conflicts
        #if conflicting_rows:
            #print(pd.DataFrame(conflicting_rows))
            #print()
        # Add the combined row to the lits of valid rows
        final_rows.append(combined_row)
print(len(final_rows))
# Combine all rows back into a DataFrame
merged_df = pd.DataFrame(final_rows)

merged_df.to_csv(joined_file_path, sep=',', index=False)

print(f"{joined_file_path}: {len(merged_df)} rows")

Index(['id', 'brand', 'model', 'date', 'year', 'display_size_inches',
       'internal_memory_in_gb', 'primary_camera_mega_pixel', 'primary_camera',
       'loud_speaker', 'gps', 'colors', 'approx_price_eur', 'battery_mah',
       'ram_in_gb', 'weight_g', 'weight_oz'],
      dtype='object')
8359
Index(['id', 'brand', 'model', 'date', 'year', 'display_size_inches',
       'internal_memory_in_gb', 'primary_camera_mega_pixel', 'primary_camera',
       'loud_speaker', 'gps', 'colors', 'approx_price_eur', 'battery_mah',
       'ram_in_gb', 'weight', 'weight_g', 'weight_oz'],
      dtype='object')
9805
Index(['id', 'brand', 'model', 'date', 'year', 'display_size_inches',
       'internal_memory_in_gb', 'primary_camera_mega_pixel', 'primary_camera',
       'loud_speaker', 'gps', 'colors', 'approx_price_eur', 'battery_mah',
       'ram_in_gb', 'weight_g', 'weight_oz', 'weight'],
      dtype='object')
118
Index(['id', 'brand', 'model', 'date', 'year', 'display_size_inches',
       'internal_mem

In [12]:
# Dropping obvious non-phones like tablets and watches
keywords = ["tablet", "watch", "ipad", "pad"]
merged_df.drop(merged_df.loc[merged_df['id'].str.contains("|".join(keywords), case=False, na=False)].index, inplace=True)

In [ ]:
# scale
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the CSV into a DataFrame
#file_path = '2017_GSM_parsed.csv'
#df = pd.read_csv(file_path)
df = merged_df.copy(deep=True)

scaled_file_path = '../2_joined_data/2017_2024_GSM_j_scaled.csv'

# Identify numerical columns, excluding 'year'
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = numerical_cols[numerical_cols != 'year']

print("scaling columns: ", list(numerical_cols))
# Apply scaling within each group while retaining original columns
group_col = 'year'

def scale_group(group):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(group[numerical_cols])
    scaled_cols = [col + '_scale' for col in numerical_cols]
    scaled_df = pd.DataFrame(scaled_data, columns=scaled_cols, index=group.index)
    for orig_col, scaled_col in zip(numerical_cols, scaled_cols):
        # Insert scaled columns right after the original columns
        col_idx = group.columns.get_loc(orig_col)
        group.insert(col_idx + 1, scaled_col, scaled_df[scaled_col])
    return group

# Apply scaling to each group by the specified column (e.g., 'year')
df = df.groupby(group_col, group_keys=False).apply(scale_group)
df = df.sort_values(['year', 'id'])

# Save the final DataFrame to a CSV file
df.to_csv(scaled_file_path, index=False)

print(len(df))
print(f"Grouped scaled data saved to {scaled_file_path}")

scaling columns:  ['display_size_inches', 'internal_memory_in_gb', 'primary_camera_mega_pixel', 'approx_price_eur', 'battery_mah', 'ram_in_gb', 'weight_g']


c:\dev\python\phoneData\.bda\Lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\dev\python\phoneData\.bda\Lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\dev\python\phoneData\.bda\Lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
c:\dev\python\phoneData\.bda\Lib\site-packages\sklearn\utils\extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\dev\python\phoneData\.bda\Lib\site-packages\sklearn\utils\extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\dev\python\phoneData\.bda\Lib\site-packages\sklearn\utils\extmath.py:1126: RuntimeWarning: inva

9672
Grouped scaled data saved to ../2_joined_data/2017_2024_GSM_j_scaled.csv
